In [24]:
# results obtained from E1
degree = 3
compression_rate = 0.3

In [25]:
%%time
from utils.data import load_ucr_dataset, load_ucr_data_short_and_medium

# load data
data = load_ucr_data_short_and_medium()
#data = load_ucr_dataset(53)
print("number of time series:", len(data.index))

number of time series: 30574
CPU times: total: 16.8 s
Wall time: 16.9 s


In [33]:
from utils.plot import add_fitted_curve_to_plot


def fit_spline(method: str, ts: [(float, float)], knots: [float], num_coeffs: int, num_outliers_removed: int,
               degree=None, plot=False, axis=None):
    match method:
        case 'PAA':
            color = 'tab:gray'
            degree = 0
            _, result = fit_max_spline(ts, knots, degree)
            fitted_curve = [evaluate_spline(knots, result, degree, x[0]) for x in ts]

        case 'PLA':
            color = 'tab:olive'
            degree = 1
            _, result = fit_max_spline(ts, knots, degree)
            fitted_curve = [evaluate_spline(knots, result, degree, x[0]) for x in ts]

        case 'L8':
            color = 'tab:pink'
            if degree is None:
                degree = 3

            _, result = fit_max_spline(ts, knots, degree)
            fitted_curve = [evaluate_spline(knots, result, degree, x[0]) for x in ts]

        case 'L8 and L1':
            color = 'tab:blue'
            if degree is None:
                degree = 3

            eps = 2e-7
            _, result = fit_max_l1_spline(ts, knots, degree, eps=eps)
            fitted_curve = [evaluate_spline(knots, result, degree, x[0]) for x in ts]

        case 'DFT':
            color = 'tab:green'
            result = fit_DFT(ts, num_coeffs)
            fitted_curve = calculate_inverse_DFT(len(ts), num_coeffs, result)

        case 'LSQ':
            color = 'tab:purple'
            if degree is None:
                degree = 3

            if num_outliers_removed > 0:
                num_xs = len(ts) + num_outliers_removed

                num_coeffs = generate_coeff_counts(num_xs, degree, [compression_rate])[0]
                knots = generate_knot_vector_from_coeff_count(degree=degree, num_coeffs=num_coeffs)

                # Alternative 1 num_coeff Ausreißer (s.u.)
                """knots = generate_knot_vector_from_coeff_count(degree=degree,
                                                              num_coeffs=num_coeffs + num_outliers_removed)"""

                # Alternative 2 (einfach neue Berechnung weglassen)

                xs = np.linspace(0, 1, num=num_xs)
                ts = replace_outliers(ts_without_outliers=ts, original_xs=xs)

            result = fit_LSQ_spline(time_series=ts, knots=knots, degree=degree)
            fitted_curve = [evaluate_spline(knots, result, degree, x[0]) for x in ts]

    max_dist = np.max(np.abs(np.array(fitted_curve) - np.array([x[1] for x in ts])))
    mse = mean_squared_error(fitted_curve, [x[1] for x in ts])
    mae = mean_absolute_error(fitted_curve, [x[1] for x in ts])

    """if plot & (axis is not None):
        add_fitted_curve_to_plot(axis, fitted_curve, max_dist, color=color, label=method)"""

    if plot:
        if axis is not None:
            #add_fitted_curve_to_plot(axis, method, knots, result, degree, max_dist, color=color, len_ts=len(ts))
            add_fitted_curve_to_plot(axis, [x[0] for x in ts], fitted_curve, max_dist, color=color, label=method)
        else:
            #add_fitted_curve_to_plot(plt, method, knots, result, degree, max_dist, color=color, len_ts=len(ts))
            add_fitted_curve_to_plot(plt, [x[0] for x in ts], fitted_curve, max_dist, color=color, label=method)

    return {'max_dist': max_dist,
            'MSE': mse,
            'MAE': mae}


# OHNE OUTLIERS, OHNE PLOTS

In [34]:
%%time
from tsfel import mean_diff
from utils.data import remove_outliers, replace_outliers
from utils.spline import generate_coeff_counts, generate_knot_vector_from_coeff_count
from utils.fit import fit_max_spline, fit_max_l1_spline, fit_DFT, calculate_inverse_DFT, fit_LSQ_spline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utils.spline import evaluate_spline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

results = []
num_rows = len(data.index)
ts_properties = []

for idx, row in data.iterrows():
    ts = row['data']
    print(f"{row['dataset']} no. {row['num']}, {round(idx / num_rows * 100, 1)}% processed")

    ts_properties.append({
        'dataset': row['dataset'],
        'num': row['num'],
        'num_data_pts': len(ts),
        'num_outliers_removed': 0,
        'mean_diff': mean_diff(ts),
        'max_fst_derivative': max(pd.Series([tup[0] for tup in ts]).diff().dropna()),
        'max_snd_derivative': max(pd.Series([tup[0] for tup in ts]).diff().diff().dropna())
    })

    num_coeffs = generate_coeff_counts(len(ts), degree, compression_ratios=[compression_rate])[0]
    knots = generate_knot_vector_from_coeff_count(degree=degree, num_coeffs=num_coeffs)

    for method in ['L8', 'L8 and L1', 'LSQ', 'PAA', 'PLA', 'DFT']:
        metrics = fit_spline(method, ts, knots, num_coeffs, 0)

        results.append({
            'dataset': row['dataset'],
            'num': row['num'],
            'num_data_pts': len(ts),
            'num_outliers_removed': 0,
            'num_coeffs': num_coeffs,
            'method': method,
            'max_dist': metrics['max_dist'],
            'MSE': metrics['MSE'],
            'MAE': metrics['MAE']
        })


Adiac no. 0, 0.3% processed
Adiac no. 1, 0.3% processed
Adiac no. 2, 0.3% processed
Adiac no. 3, 0.3% processed
Adiac no. 4, 0.3% processed
Adiac no. 5, 0.3% processed
Adiac no. 6, 0.3% processed
Adiac no. 7, 0.3% processed
Adiac no. 8, 0.4% processed


KeyboardInterrupt: 

# MIT OUTLIERS

In [35]:
"""%%time

from tsfel import mean_diff
from utils.data import remove_outliers, replace_outliers
from utils.spline import generate_coeff_counts, generate_knot_vector_from_coeff_count
from utils.fit import fit_max_spline, fit_max_l1_spline, fit_DFT, calculate_inverse_DFT, fit_LSQ_spline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utils.spline import evaluate_spline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

results = []
num_rows = len(data.index)
ts_properties = []

for idx, row in data.iterrows():
    original_time_series = row['data']
    time_series_lof = remove_outliers(row['data'])
    n_outliers = len(original_time_series) - len(time_series_lof)
    print(f"{row['dataset']} no. {row['num']}, {round(idx / num_rows * 100, 1)}% processed")

    #f, axes = plt.subplots(1, 2, sharey=True)
    #f.set_figwidth(12)

    #axes[0].set_title("Data including outliers")
    #axes[1].set_title("Data without outliers")

    for ts, num_outliers_removed in [(original_time_series, -1),(time_series_lof, n_outliers)]:
        #for ts in [original_time_series, time_series_lof]:
        #num_outliers_removed = len(original_time_series) - len(ts)
        #assert num_outliers_removed >= 0
        #print("len(ts):", len(ts))

        if num_outliers_removed == 0:
            continue

        ts_properties.append({
            'dataset': row['dataset'],
            'num': row['num'],
            'num_data_pts': len(ts),
            'num_outliers_removed': num_outliers_removed if num_outliers_removed >= 0 else 0,
            'mean_diff': mean_diff(ts),
            'max_fst_derivative': max(pd.Series([tup[0] for tup in ts]).diff().dropna()),
            'max_snd_derivative': max(pd.Series([tup[0] for tup in ts]).diff().diff().dropna())
        })

        # Alternative 1 zur Berechnung der Koeffizienten bei TS ohne Ausreißer
        #num_coeffs = generate_coeff_counts(len(original_time_series), degree, compression_ratios=[compression_rate])[0]
        #knots = generate_knot_vector_from_coeff_count(degree=degree, num_coeffs=(num_coeffs - num_outliers_removed))

        # Alternative 2 zur Berechnung der Koeffizienten bei TS ohne Ausreißer
        #num_coeffs = generate_coeff_counts(len(original_time_series), degree, compression_ratios=[compression_rate])[0]
        #knots = generate_knot_vector_from_coeff_count(degree=degree, num_coeffs=num_coeffs)

        # ursprüngliche Variante (es müssen compr_rate * n_outliers viele Punkte zusätzlich gespeichert werden)
        num_coeffs = generate_coeff_counts(len(ts), degree, compression_ratios=[compression_rate])[0]
        knots = generate_knot_vector_from_coeff_count(degree=degree, num_coeffs=num_coeffs)

        #fig,ax = plt.subplots()

        for method in ['L8', 'L8 and L1', 'LSQ', 'PAA', 'PLA', 'DFT']:
            #metrics = fit_spline(method, ts, knots, num_coeffs, num_outliers_removed)

            plot_method = False

            #if method in ['L8', 'L8 and L1']:  #'LSQ',
                #if method in ['DFT']:#
                #plot_method = True

            axis = None
            #if num_outliers_removed > 0:
                #axis = axes[1]
            #else:
                #axis = axes[0]

            metrics = fit_spline(method, ts, knots, num_coeffs, num_outliers_removed, plot=plot_method, axis=axis)

            results.append({
                'dataset': row['dataset'],
                'num': row['num'],
                'num_data_pts': len(ts),
                'num_outliers_removed': num_outliers_removed if num_outliers_removed >= 0 else 0,
                'num_coeffs': num_coeffs,
                'method': method,
                'max_dist': metrics['max_dist'],
                'MSE': metrics['MSE'],
                'MAE': metrics['MAE'],
                #'mean_diff': mean_diff(ts),
                #'fst_derivative': pd.Series([tup[0] for tup in ts]).diff().dropna(),
                #'snd_derivative':pd.Series([tup[0] for tup in ts]).diff().diff().dropna()
            })

        #axis.scatter([d[0] for d in ts], [d[1] for d in ts], color="brown", marker='.')
    
plt.subplots_adjust(bottom=0.25, top=0.95)
axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')
plt.tight_layout()
plt.show()
"""

'%%time\n\nfrom tsfel import mean_diff\nfrom utils.data import remove_outliers, replace_outliers\nfrom utils.spline import generate_coeff_counts, generate_knot_vector_from_coeff_count\nfrom utils.fit import fit_max_spline, fit_max_l1_spline, fit_DFT, calculate_inverse_DFT, fit_LSQ_spline\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error\nfrom utils.spline import evaluate_spline\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\nresults = []\nnum_rows = len(data.index)\nts_properties = []\n\nfor idx, row in data.iterrows():\n    original_time_series = row[\'data\']\n    time_series_lof = remove_outliers(row[\'data\'])\n    n_outliers = len(original_time_series) - len(time_series_lof)\n    print(f"{row[\'dataset\']} no. {row[\'num\']}, {round(idx / num_rows * 100, 1)}% processed")\n\n    #f, axes = plt.subplots(1, 2, sharey=True)\n    #f.set_figwidth(12)\n\n    #axes[0].set_title("Data including outliers")\n    #axes[1].set_title("Data without

In [36]:
import pandas as pd

pd.set_option('display.float_format', '{:.8f}'.format)

e2_results = pd.DataFrame(results).drop_duplicates()
mean_values = e2_results.groupby('method').agg({'max_dist': 'mean', 'MSE': 'mean', 'MAE': 'mean'}).reset_index()
mean_values

,method,max_dist,MSE,MAE
0,DFT,0.02077349,0.00002871,0.00376766
1,L8,0.01545623,0.00013984,0.01024594
2,L8 and L1,0.01515114,0.00005967,0.00525430
3,LSQ,0.02151217,0.00002792,0.00358524
4,PAA,0.15371210,0.01167304,0.09418549
5,PLA,0.02327365,0.00038843,0.01712178


In [37]:
"""
import pandas as pd
pd.set_option('display.float_format', '{:.8f}'.format)

# Calculate mean values for each accuracy measure separately
mean_max_dist = e2_results.groupby('method')['max_dist'].mean().reset_index()
mean_mse = e2_results.groupby('method')['MSE'].mean().reset_index()
mean_mae = e2_results.groupby('method')['MAE'].mean().reset_index()

# Merge the mean values into a single DataFrame
mean_values = mean_max_dist.merge(mean_mse, on='method').merge(mean_mae, on='method')
mean_values"""

"\nimport pandas as pd\npd.set_option('display.float_format', '{:.8f}'.format)\n\n# Calculate mean values for each accuracy measure separately\nmean_max_dist = e2_results.groupby('method')['max_dist'].mean().reset_index()\nmean_mse = e2_results.groupby('method')['MSE'].mean().reset_index()\nmean_mae = e2_results.groupby('method')['MAE'].mean().reset_index()\n\n# Merge the mean values into a single DataFrame\nmean_values = mean_max_dist.merge(mean_mse, on='method').merge(mean_mae, on='method')\nmean_values"

In [38]:
# combine results df with ts properties df for analysis later
ts_props = pd.DataFrame(ts_properties).drop_duplicates()
e2 = pd.merge(e2_results, ts_props, how='left', on=['dataset', 'num', 'num_data_pts', 'num_outliers_removed'])
e2

,dataset,num,num_data_pts,num_outliers_removed,num_coeffs,method,max_dist,MSE,MAE,mean_diff,max_fst_derivative,max_snd_derivative
0,Adiac,0,176,0,52,L8,0.01174893,0.00007591,0.00780237,-0.50000000,0.00571429,0.00000000
1,Adiac,0,176,0,52,L8 and L1,0.01174913,0.00004443,0.00508272,-0.50000000,0.00571429,0.00000000
2,Adiac,0,176,0,52,LSQ,0.01625998,0.00003225,0.00435606,-0.50000000,0.00571429,0.00000000
3,Adiac,0,176,0,52,PAA,0.15120708,0.01104580,0.09197067,-0.50000000,0.00571429,0.00000000
4,Adiac,0,176,0,52,PLA,0.01807150,0.00016987,0.01152031,-0.50000000,0.00571429,0.00000000
5,Adiac,0,176,0,52,DFT,0.01661240,0.00002817,0.00417032,-0.50000000,0.00571429,0.00000000
6,Adiac,1,176,0,52,L8,0.01288282,0.00009027,0.00849222,-0.50000000,0.00571429,0.00000000
7,Adiac,1,176,0,52,L8 and L1,0.01288302,0.00003235,0.00370505,-0.50000000,0.00571429,0.00000000
8,Adiac,1,176,0,52,LSQ,0.01805289,0.00001442,0.00251677,-0.50000000,0.00571429,0.00000000
9,Adiac,1,176,0,52,PAA,0.12701004,0.00777258,0.07775625,-0.50000000,0.00571429,0.00000000


In [39]:
df_no_outliers = e2[e2['num_outliers_removed'] > 0]
df_with_outliers = e2[e2['num_outliers_removed'] == 0]
#df_no_outliers = e2_results[e2_results['num_outliers_removed'] > 0]
#df_with_outliers = e2_results[e2_results['num_outliers_removed'] == 0]

mean_values_no_outliers = df_no_outliers.groupby('method').agg(
    {'max_dist': 'mean', 'MSE': 'mean', 'MAE': 'mean'}).reset_index()
mean_values_no_outliers.rename(columns={'max_dist': 'max_dist w/o o.', 'MSE': 'MSE w/o o.', 'MAE': 'MAE w/o o.'},
                               inplace=True)

mean_values_with_outliers = df_with_outliers.groupby('method').agg(
    {'max_dist': 'mean', 'MSE': 'mean', 'MAE': 'mean'}).reset_index()
mean_values_with_outliers.rename(columns={'max_dist': 'max_dist', 'MSE': 'MSE', 'MAE': 'MAE'}, inplace=True)

mean_values_combined = pd.merge(mean_values_with_outliers, mean_values_no_outliers, on='method', how='outer')
mean_values_combined

,method,max_dist,MSE,MAE,max_dist w/o o.,MSE w/o o.,MAE w/o o.
0,DFT,0.02077349,0.00002871,0.00376766,NaN,NaN,NaN
1,L8,0.01545623,0.00013984,0.01024594,NaN,NaN,NaN
2,L8 and L1,0.01515114,0.00005967,0.00525430,NaN,NaN,NaN
3,LSQ,0.02151217,0.00002792,0.00358524,NaN,NaN,NaN
4,PAA,0.15371210,0.01167304,0.09418549,NaN,NaN,NaN
5,PLA,0.02327365,0.00038843,0.01712178,NaN,NaN,NaN


In [40]:

import statsmodels.api as sm

# Correlation Analysis
correlation_matrix = e2[
    ['max_dist', 'MSE', 'MAE', 'num_data_pts', 'mean_diff', 'max_fst_derivative', 'max_snd_derivative']].corr()
print("Correlation Matrix:")
print(correlation_matrix)

# Regression Analysis
X = e2[['num_data_pts', 'mean_diff', 'max_fst_derivative', 'max_snd_derivative']]
y = e2['max_dist']

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())


Correlation Matrix:
                      max_dist         MSE         MAE  num_data_pts  \
max_dist            1.00000000  0.98620255  0.98704823           NaN   
MSE                 0.98620255  1.00000000  0.98387540           NaN   
MAE                 0.98704823  0.98387540  1.00000000           NaN   
num_data_pts               NaN         NaN         NaN           NaN   
mean_diff          -0.06682958 -0.06031004 -0.04429625           NaN   
max_fst_derivative         NaN         NaN         NaN           NaN   
max_snd_derivative         NaN         NaN         NaN           NaN   

                     mean_diff  max_fst_derivative  max_snd_derivative  
max_dist           -0.06682958                 NaN                 NaN  
MSE                -0.06031004                 NaN                 NaN  
MAE                -0.04429625                 NaN                 NaN  
num_data_pts               NaN                 NaN                 NaN  
mean_diff           1.00000000        

C:\Users\veron\Desktop\Bachelorarbeit_Info\Code\splineRegressionExperiments\venv\Lib\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [41]:
import pandas as pd
import statsmodels.api as sm

# Load your dataframe
# Assuming your dataframe is named 'df'

# Filter relevant columns
data = e2[['mean_diff', 'max_dist', 'method']]

# Create dummy variables for the 'method' column
data = pd.get_dummies(data, columns=['method'], drop_first=True)

# Separate independent (X) and dependent (y) variables
X = data.drop(columns=['max_dist'])
y = data['max_dist']

# Add constant for intercept term
X = sm.add_constant(X)

# Fit the regression model for each method
results = {}
method_columns = data.iloc[:, 2:]
for method_col in method_columns:
    method_name = method_col.replace('method_', '')
    method_data = data[[method_col] + ['mean_diff']]

    model = sm.OLS(y, method_data.astype(float)).fit()
    results[method_name] = model.summary()

# Print results
for method, result in results.items():
    print(f"Method: {method}")
    print(result)
    print("\n")


Method: L8
                            OLS Regression Results                            
Dep. Variable:               max_dist   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     2.879
Date:                Sun, 11 Feb 2024   Prob (F-statistic):             0.0964
Time:                        13:17:10   Log-Likelihood:                 78.081
No. Observations:                  49   AIC:                            -152.2
Df Residuals:                      47   BIC:                            -148.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
method_L8     -0.0314      0.019     -1.6

In [42]:
import pandas as pd
import statsmodels.api as sm

# Load your dataframe
# Assuming your dataframe is named 'df'

# Convert 'mean_diff' column to numeric (if not already)
e2['mean_diff'] = pd.to_numeric(e2['mean_diff'], errors='coerce')

# Drop rows with NaN values in 'mean_diff' column
df = e2.dropna(subset=['mean_diff'])

# Filter relevant columns
data = df[['mean_diff', 'max_dist', 'num_outliers_removed', 'method']]

# Create dummy variables for the 'method' column
data = pd.get_dummies(data, columns=['method'], drop_first=True)

# Separate data for num_outliers > 0 and num_outliers == 0
data_with_outliers = data[data['num_outliers_removed'] > 0]
data_without_outliers = data[data['num_outliers_removed'] == 0]

# Separate independent (X) and dependent (y) variables for each case
X_with_outliers = data_with_outliers.drop(columns=['max_dist', 'num_outliers_removed'])
y_with_outliers = data_with_outliers['max_dist']

X_without_outliers = data_without_outliers.drop(columns=['max_dist', 'num_outliers_removed'])
y_without_outliers = data_without_outliers['max_dist']

# Add constant for intercept term
X_with_outliers = sm.add_constant(X_with_outliers)
X_without_outliers = sm.add_constant(X_without_outliers)

# Fit the regression model for each case
model_with_outliers = sm.OLS(y_with_outliers, X_with_outliers.astype(float)).fit()
model_without_outliers = sm.OLS(y_without_outliers, X_without_outliers.astype(float)).fit()

# Print results
print("Results for num_outliers > 0:")
print(model_with_outliers.summary())

print("\nResults for num_outliers == 0:")
print(model_without_outliers.summary())


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
import matplotlib.pyplot as plt

# Plot mean values
plt.figure(figsize=(10, 6))

plt.bar(mean_values['method'], mean_values['max_dist'], color='b', label='Mean Max Dist')
plt.bar(mean_values['method'], mean_values['MAE'], color='g', label='Mean MAE')
plt.bar(mean_values['method'], mean_values['MSE'], color='r', label='Mean MSE')
plt.xlabel('Method')
plt.ylabel('Mean Value')
plt.title('Mean max. distance, MSE, and MAE per method')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
